<a href="https://colab.research.google.com/github/chikilivighneshshastry/colab_files/blob/main/jobdata_api_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

cookies = {
    'csrftoken': '5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}

headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9',
    'priority': 'u=1, i',
    'referer': 'https://jobdataapi.com/api/schema/swagger-ui/',
    'sec-ch-ua': '"Google Chrome";v="137", "Chromium";v="137", "Not/A)Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36',
    # 'cookie': 'csrftoken=5LSZIfHHmDnSIaZKDsN0hGYXaWQhOJGV',
}
parameters = {
    'country_code' : 'IN'
}

response = requests.get('https://jobdataapi.com/api/jobs/', cookies=cookies, headers=headers,params=parameters)

print(response)

In [ ]:
print(response.json())
data = response.json()

In [ ]:
results = data['results']
print(results)
len(results)

In [ ]:
import json
job_post = results[0]

json.dumps(job_post['cities'])

In [ ]:
import pandas as pd

df = pd.DataFrame(results)

df


In [ ]:
# @title experience_level vs salary_min

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['salary_min'].value_counts()
    for x_label, grp in df.groupby('experience_level')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('experience_level')
_ = plt.ylabel('salary_min')

In [ ]:
print(df['application_url'][0])

In [ ]:
df_city_fil = df[df['has_remote'].apply(lambda x: x)]
print(df_city_fil)

In [42]:
col = '[id, ext_id, company, title, location, types, cities, states, countries, regions, has_remote, published, description, experience_level, application_url, language, salary_min, salary_max, salary_currency]'

data = list(col.split(','))
data

['[id',
 ' ext_id',
 ' company',
 ' title',
 ' location',
 ' types',
 ' cities',
 ' states',
 ' countries',
 ' regions',
 ' has_remote',
 ' published',
 ' description',
 ' experience_level',
 ' application_url',
 ' language',
 ' salary_min',
 ' salary_max',
 ' salary_currency]']